# Установка библиотек

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Загрузим необоходимые библиотеки

In [2]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
from sklearn.metrics import classification_report, recall_score, accuracy_score
import time
import cv2
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from tqdm.notebook import tqdm
import logging
import pickle
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import subprocess as sp
from torch.utils.data import Dataset, DataLoader
import copy
from functools import lru_cache
from sklearn.model_selection import train_test_split
import catboost as cb
from PIL import ImageFilter, ImageChops

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
WORKING_DIR = '/content/drive/MyDrive/ai/video_surveillance/'
FRAME_DATA_FILE = os.path.join(WORKING_DIR, 'datasets/frame_data.pkl')
PEOPLE_DATA_FILE = os.path.join(WORKING_DIR, 'datasets/people_data.pkl')
# http://zulko.github.io/blog/2013/09/27/read-and-write-video-frames-in-python-using-ffmpeg/
FFMPEG_BIN = "ffmpeg"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Обработка csv датасета

In [5]:
train_base_df = pd.read_csv(os.path.join(WORKING_DIR, 'datasets/train/train.csv'))
test_base_df = pd.read_csv(os.path.join(WORKING_DIR, 'datasets/test/test.csv'))

In [6]:
test_base_df['dataset'] = 'test/test_fast.mp4'

In [7]:
indexes_2027 = train_base_df[train_base_df['date'].str.contains('2027-')].index
for i in indexes_2027:
    train_base_df.at[i, 'date'] = '2022' + train_base_df.at[i, 'date'][4:]

In [8]:
train_base_df['employee'] = train_base_df['employee'].replace(10, 9).replace(11, 10)

# Инициализация кэшированной даты

In [9]:
frame_data = pickle.load(open(FRAME_DATA_FILE, 'rb'))
people_data = pickle.load(open(PEOPLE_DATA_FILE, 'rb'))

# Предсказание для тестовой выборки

In [10]:
model_action = cb.CatBoostClassifier()
model_door = cb.CatBoostClassifier()

In [11]:
model_door = model_door.load_model(os.path.join(WORKING_DIR, 'model_door.cb'))
model_action = model_action.load_model(os.path.join(WORKING_DIR, 'model_action.cb'))

In [12]:
max_n_people = 26
actions = []
for i in range(len(test_base_df)):
    row = test_base_df.iloc[i]
    dataset = row['dataset']
    date = row['date']
    actions.append(0)
    if date not in frame_data[dataset].values():
        continue
    from_frame_i = min([frame_i for frame_i, date_ in frame_data[dataset].items() if date == date_])
    if from_frame_i - 10 >= 0:
        from_frame_i -= 10
    else:
        from_frame_i = 0
    to_frame_i = max([frame_i for frame_i, date_ in frame_data[dataset].items() if date == date_]) + 5
    if to_frame_i + 5 <= max(frame_data[dataset]):
        to_frame_i += 5
    else:
        to_frame_i = max(frame_data[dataset])
    n_people = []
    for frame_i in range(from_frame_i, to_frame_i + 1):
        if frame_i not in people_data[dataset]:
            n_people.append(0)
            continue
        annotations = people_data[dataset][frame_i]
        n_people.append(len(annotations))
    n_people = n_people[:max_n_people]
    for j in range(len(n_people) - 2):
        if n_people[j] != n_people[j + 1] and n_people[j] == n_people[j + 2]:
            n_people[j + 1] = n_people[j]
    
    predictig_n_people = n_people.copy()
    predictig_n_people.extend([0] * (max_n_people - len(n_people)))
    action = model_action.predict(predictig_n_people) * 10
    
    mid_i = len(n_people) // 2
    if action == 10:
        # список индексов, где появляется человек до центра
        to_middle = []
        for human_i in range(1, mid_i + 1):
            if n_people[human_i] > n_people[human_i - 1]:
                to_middle.append(human_i)
        if not any(to_middle):
            for human_i in range(mid_i + 1, len(n_people)):
                if n_people[human_i] > n_people[human_i - 1]:
                    to_middle.append(human_i)
            if not any(to_middle):
                continue
            human_i = min(to_middle)
        else:
            human_i = max(to_middle)
    else:
        # список индексов, где исчезает за дверью человек до центра
        to_middle = []
        for human_i in range(mid_i + 1, len(n_people)):
            if n_people[human_i] < n_people[human_i - 1]:
                to_middle.append(human_i)
        if not any(to_middle):
            for human_i in range(1, mid_i + 1):
                if n_people[human_i] < n_people[human_i - 1]:
                    to_middle.append(human_i)
            if not any(to_middle):
                continue
            human_i = max(to_middle)
        else:
            human_i = min(to_middle)
    
    cheking_n_people = n_people[human_i - 1:human_i + 1]
    if abs(cheking_n_people[0] - cheking_n_people[1]) != 1:
        continue
    if 0 in cheking_n_people and 1 in cheking_n_people:
        if action == 10:
            frame_i = from_frame_i + human_i
        else:
            frame_i = from_frame_i + human_i - 1
        annotation = people_data[dataset][frame_i][0]
    else:
        if action == 10:
            frame_i = from_frame_i + human_i
            annotations1 = people_data[dataset][frame_i - 1]
            annotations2 = people_data[dataset][frame_i]
        else:
            frame_i = from_frame_i + human_i - 1
            annotations1 = people_data[dataset][frame_i + 1]
            annotations2 = people_data[dataset][frame_i]
        center_annotation1 = [[annotation[0] + (annotation[2] - annotation[0]) // 2,
                               annotation[1] + (annotation[3] - annotation[1]) // 2]
                              for annotation in annotations1]
        center_annotation2 = [[annotation[0] + (annotation[2] - annotation[0]) // 2,
                               annotation[1] + (annotation[3] - annotation[1]) // 2]
                              for annotation in annotations2]
        for annotation1 in center_annotation1:
            len_to_center = []
            for annotation2 in center_annotation2:
                len_to_center.append(((annotation1[0] - annotation2[0]) ** 2 + (annotation1[1] - annotation2[1]) ** 2) ** 0.5)
            del_i = len_to_center.index(min(len_to_center))
            del annotations2[del_i]
            del center_annotation2[del_i]
        if len(center_annotation2) != 1:
            continue
        annotation = annotations2[0]
    
    action += int(model_door.predict(annotation))
    actions[-1] = action

In [13]:
test_base_df['action'] = actions
test_base_df['employee'] = 0

In [14]:
for n in range(1, 10):
    value_counts = train_base_df[train_base_df['action'].isin([n, n + 10])]['employee'].value_counts()
    if not any(value_counts.index):
        continue
    max_employee = value_counts.index[0]
    test_base_df.loc[test_base_df['action'].isin([n, n + 10]), 'employee'] = max_employee

In [15]:
test_base_df.drop(columns=['dataset', 'date']).to_csv('out.csv', index=False)